# Sports-Retail-Revenue-Optimization SQL Analysis

---


For this project. I have utilized dataset of an online sports cleaning company. This database consists of 5 relational tables info, finance, reviews, brands,traffic. By performing data transformation, product segmentation & analyzing different factors influencing revenue we aim to optimize marketing strategies and discover ways to surge revenue.

## 1. Counting Missing values

In [ ]:
%%sql
postgresql:///sports


SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN description IS NOT NULL THEN 1 ELSE 0 END) AS count_description,
    SUM(CASE WHEN listing_price IS NOT NULL THEN 1 ELSE 0 END) AS count_listing_price,
    SUM(CASE WHEN last_visited IS NOT NULL THEN 1 ELSE 0 END) AS count_last_visited
FROM
    info
JOIN
    finance USING (product_id)
JOIN
    traffic USING (product_id);





1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


## 2. Nike vs Adidas pricing

In [ ]:
%%sql
postgresql:///sports


select brand,listing_price::int,count(*)
from finance
inner join brands
using (product_id)
group by brand,listing_price
having listing_price>0
order by listing_price desc;


77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Nike,200,1
Adidas,200,8
Nike,190,2
Adidas,190,7
Nike,180,4


## 3. Product segementation based on price

In [ ]:
%%sql
postgresql:///sports

select brand,count(*),SUM(revenue) AS total_revenue ,
CASE
WHEN listing_price<42 then 'Budget'
WHEN listing_price>=42 AND listing_price<74 then 'Average'
WHEN listing_price>=74 AND listing_price<129 then 'Expensive'
WHEN listing_price>=129 then 'Elite'
END AS price_category
from finance
inner join brands as b
using (product_id)
where b.brand IS NOT NULL
group by brand,price_category
ORDER BY total_revenue DESC;

8 rows affected.


brand,count,total_revenue,price_category
Adidas,849,4626980.069999999,Expensive
Adidas,1060,3233661.060000001,Average
Adidas,307,3014316.8299999987,Elite
Adidas,359,651661.1200000002,Budget
Nike,357,595341.0199999992,Budget
Nike,82,128475.59000000003,Elite
Nike,90,71843.15000000004,Expensive
Nike,16,6623.5,Average


## 4. Average discount by brand

In [ ]:
%%sql
postgresql:///sports



select brand,avg(discount)*100 as average_discount
from finance
inner join brands
using (product_id)
where brand IS NOT NULL
group by brand

2 rows affected.


brand,average_discount
Nike,0.0
Adidas,33.452427184465606


## 5. Correlation between revenue and reviews

In [ ]:
%%sql

postgresql:///sports

select corr(reviews,revenue) as review_revenue_corr
from finance
join reviews
using (product_id)

1 rows affected.


review_revenue_corr
0.6518512283481301


## 6. Effect of description length on ratings & reviews

In [ ]:
%%sql
postgresql:///sports



select trunc(length(description),-2) as description_length, round(avg(rating::numeric),2) as average_rating
from info
inner join reviews using (product_id)
where description IS NOT NULL
group by description_length
order by description_length;

7 rows affected.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


## 7. Reviews  by month and brand

In [ ]:
%%sql
postgresql:///sports



select brand, extract(month from last_visited) as month , count(*) as num_reviews
from traffic
join brands using (product_id)
join reviews using (product_id)
where brand IS NOT NULL AND last_visited is not null
group by brand,month
order by brand,month

24 rows affected.


brand,month,num_reviews
Adidas,1,253
Adidas,2,272
Adidas,3,269
Adidas,4,180
Adidas,5,172
Adidas,6,159
Adidas,7,170
Adidas,8,189
Adidas,9,181
Adidas,10,192


## 8. Footwear Product performance

In [ ]:
%%sql
postgresql:///sports



with footwear as (
select description,revenue
from info
inner join finance
using (product_id)
where (description ILIKE '%shoe%' OR
description ILIKE '%trainer%' OR
description ILIKE '%foot%') and
DESCRIPTION is not null
)

select count(*) AS num_footwear_products,
PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY revenue) as median_footwear_revenue
from footwear;



1 rows affected.


num_footwear_products,median_footwear_revenue
2700,3118.36


## 9. Clothing product performance

In [ ]:
%%sql
postgresql:///sports


with footwear as (
select description,revenue
from info
inner join finance
using (product_id)
where (description ILIKE '%shoe%' OR
description ILIKE '%trainer%' OR
description ILIKE '%foot%') and
DESCRIPTION is not null
)

select count(*) as num_clothing_products,
PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY revenue) as median_clothing_revenue
from info
inner join finance using (product_id)
where description NOT ILIKE '%shoe%' AND
      description NOT ILIKE '%trainer%' AND
      description NOT ILIKE '%foot%';


1 rows affected.


num_clothing_products,median_clothing_revenue
417,503.82
